## 安装，配置使用 CMake 工具链

按照 [官方教程](https://github.com/ngtcp2/ngtcp2) 的前一半安装好 `OpenSSL_1_1_1q+quic` 和 `nghttp3`

``` bash
$ git clone --depth 1 -b OpenSSL_1_1_1q+quic https://github.com/quictls/openssl
$ cd openssl
$ # For Linux
$ ./config enable-tls1_3 --prefix=$PWD/build
$ make -j$(nproc)
$ make install_sw
$ cd ..
$ git clone https://github.com/ngtcp2/nghttp3
$ cd nghttp3
$ autoreconf -i
$ ./configure --prefix=$PWD/build --enable-lib-only
$ make -j$(nproc) check
$ make install
$ cd ..
$ git clone https://github.com/ngtcp2/ngtcp2
$ cd ngtcp2
```

在 VSCode 中配置 CMake ，重点配置 CMake 的环境变量 PKG_CONFIG_PATH 为刚刚安装好的两个库，`settings.json` 如下

``` json
{
    "cmake.configureOnOpen": true,
    "cmake.environment": {
        "PKG_CONFIG_PATH": "${workspaceFolder}/../openssl/build/lib/pkgconfig:${workspaceFolder}/../nghttp3/build/lib/pkgconfig",
    },
    "files.associations": {
        "string_view": "cpp"
    },
}
```

即可使用 CMake 编译项目

## 使用 examples/ 中的客户端和服务端

`examples/server.cc` 似乎实现了一个使用 http3 的服务器，可以使用 `examples/client.cc` 连接与访问

默认情况下编译好的可执行文件在 `build/` 中对应目录下

### 服务端

先用 OpenSSL 签发证书，参考：<https://zhuanlan.zhihu.com/p/367630416>

生成 PRIVATE_KEY_FILE 为 `server.key`，CERTIFICATE_FILE 为 `server.crt`

再生成一些用于传输的文件，比如生成一个 1M 大小的全零文件 `1M.file`

``` bash
dd if=/dev/zero of=1M.file bs=1M count=1
```

如下命令运行服务端

- '*' 表示监听所有地址
- 2333 为端口
- `--qlog-dir` 为 qlog 日志目录，目录下文件名为 Source Connection ID，qlog 日志文件可以在这个网站上可视化: https://qvis.quictools.info/#/files
- `-q` 关闭调试输出（原程序中调试信息过多），可以修改源码直接输出需要的信息

In [ ]:
!./build/examples/server '*' 2333 ./build/examples/server.key ./build/examples/server.crt --qlog-dir ./build/examples/server_log/ -q

### 客户端

- `--download` 为文件下载目录
- 末尾为要访问的 URL

In [ ]:
!./build/examples/client 127.0.0.1 2333  --qlog-dir ./build/examples/client_log/ --download ./build/examples/download/ https://127.0.0.1:2333/build/examples/data/1M.file

## 使用 Mahimahi 控制网络条件

[Mahimahi](http://mahimahi.mit.edu/)

- `mm-delay` 设置延迟，如 `mm-delay 100` 设置 100ms 延迟
- `mm-loss` 设置丢包率，如 `mm-loss uplink 0.2 mm-loss downlink 0.3` 设置上行 20% 丢包率，下行 30% 丢包率
- `mm-link` 使用 trace 文件模拟网络速率，如 `mm-link /usr/share/mahimahi/traces/Verizon-LTE-short.up /usr/share/mahimahi/traces/Verizon-LTE-short.down` 使用对应的上下行 trace 文件。文档里没说的是 `mm-link` 还有参数可限制缓冲区大小： `--downlink-queue=droptail --downlink-queue-args=bytes=???`

Mahimahi 仓库也提供了一些实际网络测速得到的 trace 文件：<https://github.com/ravinet/mahimahi/tree/master/traces>

trace 文件中每一个数字代表在这个时间点可以通过一个包，例如：

``` txt
0
0
```

表示

这些指令可以结合使用，相当于多层包装，例如：

``` bash
mm-delay 100 mm-loss uplink 0.2 mm-loss downlink 0.2 mm-link /usr/share/mahimahi/traces/Verizon-LTE-short.up /usr/share/mahimahi/traces/Verizon-LTE-short.down
```

在 Mahimahi container 中，访问本机架设的服务需要使用 "base" container 的 IP 变量 $MAHIMAHI_BASE 作为访问的地址，使用 127.0.0.1 访问的是当前容器。客户端使用可以改成如下命令：

``` bash
./build/examples/client $MAHIMAHI_BASE 2333  --qlog-dir ./build/examples/client_log/ --download ./build/examples/download/ https://$MAHIMAHI_BASE:2333/build/examples/data/1M.file
```

## 20220718 实验

初步熟悉代码，在源码中对 CUBIC 初步修改，直接记录上一次连接最后的 CWnd，在下一次连接时直接使用。

服务端：

``` bash
./build/examples/server '*' 2333 ./build/examples/server.key ./build/examples/server.crt --qlog-dir ./build/examples/server_log/ -q
```

客户端：

``` bash
mm-delay 100 mm-link --downlink-queue=droptail --downlink-queue-args=bytes=??? /usr/share/mahimahi/traces/Verizon-LTE-short.up /usr/share/mahimahi/traces/Verizon-LTE-short.down
```

``` bash
./build/examples/client $MAHIMAHI_BASE 2333 --download ./build/examples/download/ https://$MAHIMAHI_BASE:2333/build/examples/data/1M.file -q
```

## 20220801 实验

### 目标

- [] 将修改后的拥塞控制算法提取为单独的文件
- [] CUBIC 初始化时即设置 CWnd
- [] 解析连接 IP 地址
- [] 命令行指定测试参数/自动化测试脚本
